# Process new data from liulin

In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
import xml.etree.ElementTree as ET
from glob import glob
import datetime

## 0. Replace strings in the navigation data files (XML parser helper)

In [48]:
# This is necesary to run only for data from year 2015

# process all directories
for directory in glob('./data/*/'): # search for measurement runs
    for file in glob(directory + 'Navigation data/*.xml'): #search for navigation data
        print file
        #fileoutput = './output/' + file.split('/')[4]

        with open(file) as f:
            newText=f.read().replace('" XML_LMARK "', '<')
        with open(file, "w") as f:
            f.write(newText)

        with open(file) as f:
            newText=f.read().replace('" XML_RMARK "', '>')
        with open(file, "w") as f:
            f.write(newText)

        with open(file) as f:
            newText=f.read().replace('" XML_RMARK"', '>')
        with open(file, "w") as f:
            f.write(newText)
            
        with open(file) as f:
            newText=f.read().replace('" XML_HEADER "', '<?xml version="1.0" encoding="utf-8"?>')
        with open(file, "w") as f:
            f.write(newText)

./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_190_2016-05-19.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_190_2016-05-22.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_190_2016-05-26.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_190_2016-04-21.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_190_2016-03-19.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_191_2016-05-17.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_191_2016-04-15.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_190_2016-03-24.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_191_2016-05-24.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_191_2016-03-06.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA_191_2016-05-22.xml
./data/45. (03. 04. 2016-27. 05. 2016)/Navigation data/RAD_OK-YBA

# 1. Calculation of doserate and concatenate dosimetric data with navigation data for all subdirectories

In [11]:
def round_dt(dt): # round time to whole minutes
    dt = dt + datetime.timedelta(seconds=30, microseconds=999999)
    dt = dt.replace(second=0, microsecond=0)
    return dt


# process all directories

for directory in glob('./data/*/'): # search for measurement runs
    for filename in glob( directory + 'Liulin/*.y*'): # search for Liuline files

        print '----------------------------------------------------------------------'
        print directory
        # parse Run number from the directory name
        run = directory.split('/')
        run = run[2].split('(')
        run = run[0].split('.')
        
        # extract time from Liulin filename
        path = filename.split('/')
        time = path[-1].split('.')
        date_object = datetime.datetime.strptime(time[0], '%y%m%d%H%M')
        print date_object
        print 'RUN ', run[0]
        
        # read data from Liulin 
        liulin = pd.read_csv(filename, header = None, sep = ' ', skiprows = 1) # read Liulin data Y
        liulin = liulin.drop(256, axis=1) # delete last empty column
        infile = open(filename, 'r')
        header = infile.readline()
        print header # print data header
        exposition = header.split(' ')[4].split('[')[0] # extract exposition time  
        exposition_val = float(exposition)
        exposition += 'S'
        
        # sum energy
        energy_scale = np.linspace(0.0407, 20.7977, 256)    # 0.0814 MeV per channel
        uGy_const = 1.602e-7 / 1.398e-4 * (3600/exposition_val)     # to uGy/h for given exposition
        liulin_energy = liulin
        liulin_energy = liulin_energy.multiply(energy_scale)
        liulin[256] = liulin_energy.sum(axis=1) * uGy_const # DSi
        liulin[257] = liulin_energy.ix[:,0:11].sum(axis=1) * uGy_const * 1.7 + liulin_energy.ix[:,12:255].sum(axis=1) * uGy_const * 6.1 # H
       
        # index liulin data (compute time)
        liulin_data = pd.DataFrame(index = pd.date_range(date_object, freq=exposition, periods=len(liulin)).tolist(), data = liulin.as_matrix())


    # Parse Navigation data
    nav_data = pd.DataFrame()
    
    for file in glob(directory + 'Navigation data/*.xml'): #search for navigation data
        #print file
        try:
            tree = ET.parse(file) # parse XML
            values = tree.findall('./FlightTrack/Position')
    
            data = pd.DataFrame({
                             #'time' : round_dt(datetime.datetime.strptime(value.find('Time').text, "%Y-%m-%d %H:%M:%S")), 
                             #'line' : tree.find('./FlightInfo/FlightLine').text, # !!! IMZ
                             'from' : tree.findtext('./FlightInfo/Org',default='XXX'),
                             'to' : tree.findtext('./FlightInfo/Dst',default='XXX'),
                             'lon' : [value.find('Longtitude').text for value in values],
                             'lat' : [value.find('Latitude').text for value in values],
                             'alt' : [value.find('Baralt').text for value in values],
                             }, index=[round_dt(datetime.datetime.strptime(value.find('Time').text, "%Y-%m-%d %H:%M:%S")) for value in values])
        
            nav_data = pd.concat([data,nav_data]) # add nav_data from one file
        except:
            print 'ERROR parse XML: ' + file
            print sys.exc_info()
            continue

    # Merge data
    data = nav_data.join(liulin_data) # merge liulin data with navigation data
    data.rename(columns={256: 'DSi', 257: 'H'}, inplace=True)
    data = data.dropna(subset=[7]) # delele NaN lines
    #data.index.name = 'time'
    #data = data.sort_index() # sort by time
    try:
        data['flight'] = '*' + data['from'] + data['to']
        data['E'] = 0.0
    except:
        print 'ERROR no data:', directory
        print
        continue
    
    # rearrange culomns    
    out = data[['flight','lat','lon','alt','DSi','H', 'E']+list(xrange(256))]
    out.reset_index(inplace=True)
    out.columns = ['date','Flight','lat','lon','alt','DSi[uGy/h]','H*(10)[uSv/h]','E(CARI)[uSv/h]']+list(xrange(1,257))
    out.set_index('date',inplace=True)
    out.sort_index(inplace=True) # sort by time

    # save
    out.to_csv('./output/Run' + run[0] + '.txt', sep='\t') # save merged data to output directory



----------------------------------------------------------------------
./data/45. (03. 04. 2016-27. 05. 2016)/
2016-03-04 22:49:00
RUN  45
MDU-07  EXPOSITION = 300[sec]

----------------------------------------------------------------------
./data/43. (18. 03. 2015-20. 06. 2015)/
2015-03-18 06:55:00
RUN  43
MDU-07  EXPOSITION = 300[sec]

----------------------------------------------------------------------
./data/47. (24. 9. 2016 -1. 11. 2016)/
2016-09-24 05:51:00
RUN  47
MDU-07  EXPOSITION = 60[sec]

----------------------------------------------------------------------
./data/48. (17. 03. 2017-16. 04 2017)/
2017-03-17 16:13:00
RUN  48
MDU-07  EXPOSITION = 60[sec]

----------------------------------------------------------------------
./data/44. (29. 06. 2015-24. 09. 2015)/
2015-06-29 10:18:00
RUN  44
MDU-07  EXPOSITION = 300[sec]

----------------------------------------------------------------------
./data/46. (01. 06. 2016-30. 06. 2016)/
2016-06-01 10:18:00
RUN  46
MDU-07  EXPOSIT

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# 2. Concatenate Runs files to the one file

In [ ]:
import fileinput
import glob

filenames = glob.glob("./data/Run*.txt")
filenames = sorted(filenames)
print filenames
outfilename = './data/AllRun.txt'

with open(outfilename, 'w') as fout:
    line = 'date	flight	lat	lon	alt	DSi	H	E	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	33	34	35	36	37	38	39	40	41	42	43	44	45	46	47	48	49	50	51	52	53	54	55	56	57	58	59	60	61	62	63	64	65	66	67	68	69	70	71	72	73	74	75	76	77	78	79	80	81	82	83	84	85	86	87	88	89	90	91	92	93	94	95	96	97	98	99	100	101	102	103	104	105	106	107	108	109	110	111	112	113	114	115	116	117	118	119	120	121	122	123	124	125	126	127	128	129	130	131	132	133	134	135	136	137	138	139	140	141	142	143	144	145	146	147	148	149	150	151	152	153	154	155	156	157	158	159	160	161	162	163	164	165	166	167	168	169	170	171	172	173	174	175	176	177	178	179	180	181	182	183	184	185	186	187	188	189	190	191	192	193	194	195	196	197	198	199	200	201	202	203	204	205	206	207	208	209	210	211	212	213	214	215	216	217	218	219	220	221	222	223	224	225	226	227	228	229	230	231	232	233	234	235	236	237	238	239	240	241	242	243	244	245	246	247	248	249	250	251	252	253	254	255	256\n'    
    fout.write(line)
    for line in fileinput.input(filenames):
        if line[0] != 'd':
            fout.write(line)
    